<a href="https://colab.research.google.com/github/GDharan10/Project8_ClassificationModelForFinancialStatements/blob/main/Classification_Model_For_Financial_Statementsipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [2]:
import os
import pandas as pd
import re

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data collection**

In [4]:
# Function to clean the text
def clean_text(text):
    # Remove specified substrings only if they are standalone words
    substrings_to_remove = ['A', 'B', 'C', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'ii', 'iii', 'iv', 'v', 'vi']
    for substr in substrings_to_remove:
        text = re.sub(r'\b{}\b'.format(substr), '', text)
    # Remove non-alphanumeric characters except spaces
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Folder path containing HTML files
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Balance Sheets"

# Initialize an empty list to store DataFrames
dfs_list = []

# Process each HTML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        file_path = os.path.join(folder_path, filename)
        # Read HTML tables into a list of DataFrame objects
        dfs = pd.read_html(file_path)
        # Assuming the desired table is the first one
        table_df = dfs[0]
        # Extract text from the second column
        extracted_text = ' '.join(str(cell) for cell in table_df[1] if pd.notnull(cell))
        # Document name
        document_name = "Balance Sheets"
        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)
        # Create a DataFrame with cleaned text and document name
        df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': [document_name]})
        # Append the DataFrame to the list
        dfs_list.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
BalanceSheets_df = pd.concat(dfs_list, ignore_index=True)

# Display the final DataFrame
print(BalanceSheets_df)


                                        Extracted_text   document_name
0    Particulars No ASSETS NonCurrent Assets Proper...  Balance Sheets
1                         STANDALONE March Audited III  Balance Sheets
2    GRINDWELL NORTON LIMITED Statement of Standalo...  Balance Sheets
3    ASSETS NonCurrent Assets Property Plant and Eq...  Balance Sheets
4                                  N SRBCCO LLP MUMBAI  Balance Sheets
..                                                 ...             ...
265  AUDITED CONSOLIDATED STATEMENT OF ASSETS AND L...  Balance Sheets
266  Notes to standalone I ml AS financial results ...  Balance Sheets
267  Particulars ASSETS NonCurrent Assets Property ...  Balance Sheets
268  Particulars ASSETS Noncurrent assets Property ...  Balance Sheets
269  Particulars Assets NonCurrent Assets Property ...  Balance Sheets

[270 rows x 2 columns]


In [ ]:
BalanceSheets_df.Extracted_text.unique()

array(['Particulars No ASSETS NonCurrent Assets Property Plant and Equipment Capital WorkInProgress Investment Property Goodwill Other Intangible Assets f Intangible Assets Under Development g Investment in an Associate h Financial Assets Investments Loans Others Deferred Tax Assets Net j Other NonCurrent Assets Sub Total NonCurrent Assets Current Assets Inventories Financial Assets Investments Trade Receivables Cash and Cash Equivalents iv Bank Balance other than above v Others Current Tax Assets Net Other Current Assets Sub Total Current Assets TOTAL ASSETS EQUITY AND LIABILITIES EQUITY Equity Share Capital Other Equity Non Controlling Interest Sub Total Equity LIABILITIES NonCurrent Liabilities Financial Liabilities Borrowings Trade Payable Other Financial Liabilities Provisions Deferred Tax Liabilities Net Other NonCurrent Liabilities Sub Total NonCurrent Liabilities Current Liabilities Financial Liabilities Borrowings Trade Payable Other Financial Liabilities Other Current Liabili

In [6]:
# Define the folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Cash Flow"

# Function to extract text from HTML files and clean it
def extract_and_clean_text(file_path):
    # Read HTML tables into a list of DataFrame objects
    dfs = pd.read_html(file_path)

    # Combine text from all tables into a single string
    extracted_text = ' '.join(' '.join(str(cell) for cell in table_df[0] if pd.notnull(cell)) for table_df in dfs)

    # Remove unwanted characters and symbols
    cleaned_text = re.sub(r'[^\w\s]', '', extracted_text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# List to store extracted data
data = []

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".html"):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Extract and clean text from the file
        extracted_text = extract_and_clean_text(file_path)

        # Add extracted text and document name to the data list
        data.append({'Extracted_text': extracted_text, 'document_name': "Cash Flow"})

# Create a DataFrame from the extracted data
CashFlow_df = pd.DataFrame(data)

# Display the DataFrame
print(CashFlow_df)

                                       Extracted_text document_name
0   PARTICULARS Net profit before tax ADD Deprecia...     Cash Flow
1   Cash flows from financing activities Decrease ...     Cash Flow
2   CASH FLOW FROM OPERATING ACTIVITIES Profit bef...     Cash Flow
3                               DESCRIPTION A A A B C     Cash Flow
4   Particulars Cash flow from financing activitie...     Cash Flow
5                                 Particulars A B C D     Cash Flow
6   A Cash flow from operating activities Profit b...     Cash Flow
7                                               A B C     Cash Flow
8                                               A B C     Cash Flow
9   Particulars Cash flow From operating activitie...     Cash Flow
10  Particulars Cash Flow from operating activitie...     Cash Flow
11                                         C C Note 1     Cash Flow
12  B CASH FLOW FROM INVESTING ACTIVITIES Fixed As...     Cash Flow
13  CASH FLOW FROM FINANCING ACTIVITIES Dividend

In [7]:
CashFlow_df.Extracted_text.unique()

array(['PARTICULARS Net profit before tax ADD Depreciation ADD Misc Expenses ADD Interest Less Interest received Operating Profit Increase in inventories Increase in debtors Increase in loan advances IncreaseDecrease in current liabilities Cash generated from operations less Tax expense Less Interest paid Cash from operations Increase in Investments Add RD inprocess materials Add Additions to Fixed Assets Add Increase in Capital Work in progress Add RD Capital Expenditure Net Cash Used in Investing Receipts from share capital Receipts from new borrowings Net Cash Used Financing Activity Net cash Flow Opening Balance Net cash Flow Cash and Cash Equivalents',
       'Cash flows from financing activities Decrease Increase of principal under capital lease obligations net Proceeds from short term borrowings Repayment of short term borrowings Proceeds from long term debt Repayment of long term debt Buyback of equity shares including transaction cost Payment for deferred consideration on busi